In [16]:
import json
import pandas as pd
import librosa
from sklearn.metrics import classification_report
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
import numpy as np

In [62]:
df_train = pd.read_csv("train_data.csv")
df_test = pd.read_csv("test_data.csv")

In [63]:
df_train=df_train.drop(['pitch0','pitch1','pitch2','fb00','fb01','fb02'],axis=1)
#loading fb0 
df_train['fb0']="/Users/samhitha/Documents/USC MsCS/Third Sem/CSCI-535 Multimodal/Project/fb3_json/sw" + df_train['file'].str.replace(".txt","")+"-"+df_train['Channel']+".json"
#loading pitch
df_train['pitch']="/Users/samhitha/Documents/USC MsCS/Third Sem/CSCI-535 Multimodal/Project/pitch_json/sw" + df_train['file'].str.replace(".txt","")+"-"+df_train['Channel']+".json"

In [64]:
df_test.drop(['pitch0','pitch1','pitch2','fb00','fb01','fb02'],axis=1)
#loading fb0 
df_test['fb0']="/Users/samhitha/Documents/USC MsCS/Third Sem/CSCI-535 Multimodal/Project/fb3_json/sw" + df_test['file'].str.replace(".txt","")+"-"+df_test['Channel']+".json"
#loading pitch
df_test['pitch']="/Users/samhitha/Documents/USC MsCS/Third Sem/CSCI-535 Multimodal/Project/pitch_json/sw" + df_test['file'].str.replace(".txt","")+"-"+df_test['Channel']+".json"


In [65]:
df_train

,Unnamed: 0,Channel,Utterance,file,start,end,Label,fb0,pitch
0,24551,A,otherwise theyre pretty smelly,4019.txt,220.071750,221.542875,sv,/Users/samhitha/Documents/USC MsCS/Third Sem/C...,/Users/samhitha/Documents/USC MsCS/Third Sem/C...
1,23101,B,and i think a lot of their friends feel the sa...,2709.txt,432.140875,434.443250,sd,/Users/samhitha/Documents/USC MsCS/Third Sem/C...,/Users/samhitha/Documents/USC MsCS/Third Sem/C...
2,27063,B,well now so if you were going to have a dinner...,3506.txt,3.879375,7.575625,qw,/Users/samhitha/Documents/USC MsCS/Third Sem/C...,/Users/samhitha/Documents/USC MsCS/Third Sem/C...
3,20702,A,they uh,3228.txt,55.163000,55.805875,%,/Users/samhitha/Documents/USC MsCS/Third Sem/C...,/Users/samhitha/Documents/USC MsCS/Third Sem/C...
4,5976,B,i i dont feel like theyre a benefit to society...,3247.txt,40.875875,44.615500,sv,/Users/samhitha/Documents/USC MsCS/Third Sem/C...,/Users/samhitha/Documents/USC MsCS/Third Sem/C...
...,...,...,...,...,...,...,...,...,...
34235,9142,B,latex paint,3431.txt,76.686625,77.410500,qy,/Users/samhitha/Documents/USC MsCS/Third Sem/C...,/Users/samhitha/Documents/USC MsCS/Third Sem/C...
34236,29158,B,where are you calling from by the way texas,2627.txt,164.720625,440.489500,fc,/Users/samhitha/Documents/USC MsCS/Third Sem/C...,/Users/samhitha/Documents/USC MsCS/Third Sem/C...
34237,8837,B,okay,2263.txt,0.764875,1.559250,"fo_o_fw_""_by_bc",/Users/samhitha/Documents/USC MsCS/Third Sem/C...,/Users/samhitha/Documents/USC MsCS/Third Sem/C...
34238,28557,A,but i mean they do,4733.txt,93.566375,94.416375,sd,/Users/samhitha/Documents/USC MsCS/Third Sem/C...,/Users/samhitha/Documents/USC MsCS/Third Sem/C...


In [66]:
def makedataset(arg,df):
    oldp=""
    final=[]
    for index, row in df.iterrows():
        if(oldp!=row['pitch']):
            fp=open(row['pitch'])
            filep = json.load(fp)
            ff=open(row['fb0'])
            filef = json.load(ff)

        start = int(row['start']*100)
        end = int(row['end']*100)

        listp = filep[start:end+1]
        listf = filef[start:end+1]
        
        if arg =='mean':
            
            max0p = sum(l[0] for l in listp)
            max1p= sum(l[1] for l in listp)
            max2p = sum(l[2] for l in listp)

            max0f = sum(l[0] for l in listf)
            max1f= sum(l[1] for l in listf)
            max2f = sum(l[2] for l in listf)

            n=len(listp)

            tempp = [row['Channel'],row['Utterance'],row['file'],row['start'],row['end'],max0p/n,max1p/n,max2p/n,max0f/n,max1f/n,max2f/n,row['Label']]
        
        elif arg == 'max':
            max0p = max(l[0] for l in listp)
            max1p= max(l[1] for l in listp)
            max2p = max(l[2] for l in listp)

            max0f = max(l[0] for l in listf)
            max1f= max(l[1] for l in listf)
            max2f = max(l[2] for l in listf)

            tempp = [row['Channel'],row['Utterance'],row['file'],row['start'],row['end'],max0p,max1p,max2p,max0f,max1f,max2f,row['Label']]
            
        elif arg == '90per':
            feature1p=[]
            feature2p=[]
            feature3p=[]
            for l in listp:
                feature1p.append(l[0])
                feature2p.append(l[1])
                feature3p.append(l[2])
            
            feature1f=[]
            feature2f=[]
            feature3f=[]
            for l in listf:
                feature1f.append(l[0])
                feature2f.append(l[1])
                feature3f.append(l[2])

            tempp = [row['Channel'],row['Utterance'],row['file'],row['start'],row['end'],np.percentile(feature1p,90),np.percentile(np.array(feature2p),90),np.percentile(np.array(feature3p),90),np.percentile(np.array(feature1f),90),np.percentile(np.array(feature2f),90),np.percentile(np.array(feature3f),90),row['Label']]
          
        elif arg == '95per':
            feature1p=[]
            feature2p=[]
            feature3p=[]
            for l in listp:
                feature1p.append(l[0])
                feature2p.append(l[1])
                feature3p.append(l[2])
            
            feature1f=[]
            feature2f=[]
            feature3f=[]
            for l in listf:
                feature1f.append(l[0])
                feature2f.append(l[1])
                feature3f.append(l[2])

            tempp = [row['Channel'],row['Utterance'],row['file'],row['start'],row['end'],np.percentile(feature1p,95),np.percentile(np.array(feature2p),95),np.percentile(np.array(feature3p),95),np.percentile(np.array(feature1f),95),np.percentile(np.array(feature2f),95),np.percentile(np.array(feature3f),95),row['Label']]
  
        final.append(tempp)
        oldp= row['pitch']

    data = pd.DataFrame(final,columns=['Channel','Utterance','file','start','end','pitch0','pitch1','pitch2','fb00','fb01','fb02','Label'])
    return data

In [67]:
train = makedataset("90per",df_train)
test =  makedataset("90per",df_test)

In [43]:
train_utterances=train.iloc[:,-7:-1]
y_train=train.iloc[:,-1]
test_utterances=test.iloc[:,-7:-1]
y_test=test.iloc[:,-1]

In [44]:
model = Perceptron()

model.fit(train_utterances, y_train)
pred = model.predict(test_utterances)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

,precision,recall,f1-score,support
%,0.136364,0.070312,0.092784,128.000000
^2,0.000000,0.000000,0.000000,7.000000
^g,0.000000,0.000000,0.000000,1.000000
^h,0.000000,0.000000,0.000000,6.000000
^q,0.000000,0.000000,0.000000,10.000000
aa,0.000000,0.000000,0.000000,101.000000
aap_am,0.000000,0.000000,0.000000,1.000000
ad,0.000000,0.000000,0.000000,11.000000
ar,0.000000,0.000000,0.000000,4.000000
arp_nd,0.000000,0.000000,0.000000,3.000000


In [45]:
model = SVC(class_weight='balanced')

model.fit(train_utterances, y_train)
pred = model.predict(test_utterances)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(res

,precision,recall,f1-score,support
%,0.294118,0.039062,0.068966,128.000000
^2,0.000000,0.000000,0.000000,7.000000
^g,0.000000,0.000000,0.000000,1.000000
^h,0.000000,0.000000,0.000000,6.000000
^q,0.000000,0.000000,0.000000,10.000000
aa,0.000000,0.000000,0.000000,101.000000
aap_am,0.000000,0.000000,0.000000,1.000000
ad,0.000000,0.000000,0.000000,11.000000
ar,0.000000,0.000000,0.000000,4.000000
arp_nd,0.000000,0.000000,0.000000,3.000000


In [46]:
model = LogisticRegression()
model.fit(train_utterances, y_train)
pred = model.predict(test_utterances)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWar

,precision,recall,f1-score,support
%,0.500000,0.007812,0.015385,128.000000
^2,0.000000,0.000000,0.000000,7.000000
^g,0.000000,0.000000,0.000000,1.000000
^h,0.000000,0.000000,0.000000,6.000000
^q,0.000000,0.000000,0.000000,10.000000
aa,0.000000,0.000000,0.000000,101.000000
aap_am,0.000000,0.000000,0.000000,1.000000
ad,0.000000,0.000000,0.000000,11.000000
ar,0.000000,0.000000,0.000000,4.000000
arp_nd,0.000000,0.000000,0.000000,3.000000


In [47]:
model = LinearSVC(class_weight='balanced')

model.fit(train_utterances, y_train)
pred = model.predict(test_utterances)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

,precision,recall,f1-score,support
%,0.250000,0.015625,0.029412,128.000000
^2,0.000000,0.000000,0.000000,7.000000
^g,0.000000,0.000000,0.000000,1.000000
^h,0.000000,0.000000,0.000000,6.000000
^q,0.000000,0.000000,0.000000,10.000000
aa,0.000000,0.000000,0.000000,101.000000
aap_am,0.000000,0.000000,0.000000,1.000000
ad,0.000000,0.000000,0.000000,11.000000
ar,0.000000,0.000000,0.000000,4.000000
arp_nd,0.000000,0.000000,0.000000,3.000000


In [48]:
from sklearn import tree

model = tree.DecisionTreeClassifier()
model.fit(train_utterances, y_train)
pred = model.predict(test_utterances)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

,precision,recall,f1-score,support
%,0.152174,0.164062,0.157895,128.000000
^2,0.000000,0.000000,0.000000,7.000000
^g,0.000000,0.000000,0.000000,1.000000
^h,0.000000,0.000000,0.000000,6.000000
^q,0.000000,0.000000,0.000000,10.000000
aa,0.088608,0.069307,0.077778,101.000000
aap_am,0.000000,0.000000,0.000000,1.000000
ad,0.000000,0.000000,0.000000,11.000000
ar,0.000000,0.000000,0.000000,4.000000
arp_nd,0.000000,0.000000,0.000000,3.000000


In [49]:
from sklearn.neighbors import KNeighborsClassifier
acc=[]
for k in range(1,50):
    model = KNeighborsClassifier(n_neighbors = k)
    model.fit(train_utterances, y_train)
    pred = model.predict(test_utterances)
    pred = classification_report(y_test, pred, output_dict=True)
    report = pd.DataFrame(pred).transpose()
    acc.append(report.iloc[-3,0])


/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [50]:
acc

[0.2978369384359401,
 0.26622296173044924,
 0.2995008319467554,
 0.3699389905712701,
 0.3860232945091514,
 0.3998890737659456,
 0.4054353854686633,
 0.4120909595119246,
 0.42207432057681643,
 0.4292845257903494,
 0.4348308374930671,
 0.44093178036605657,
 0.4437049362174154,
 0.43982251802551303,
 0.4420410427066001,
 0.44315030504714364,
 0.4453688297282307,
 0.454242928452579,
 0.45368829728230725,
 0.4525790349417637,
 0.45535219079312256,
 0.45535219079312256,
 0.45535219079312256,
 0.45757071547420963,
 0.461453133666112,
 0.4564614531336661,
 0.45923460898502494,
 0.4597892401552967,
 0.45757071547420963,
 0.4564614531336661,
 0.4603438713255685,
 0.4620077648363838,
 0.45923460898502494,
 0.46089850249584025,
 0.4603438713255685,
 0.4620077648363838,
 0.4603438713255685,
 0.45923460898502494,
 0.46089850249584025,
 0.4597892401552967,
 0.4597892401552967,
 0.45867997781475317,
 0.46089850249584025,
 0.4597892401552967,
 0.46089850249584025,
 0.4597892401552967,
 0.45923460898502

In [51]:
model = KNeighborsClassifier(n_neighbors = 25)
model.fit(train_utterances, y_train)
pred = model.predict(test_utterances)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

,precision,recall,f1-score,support
%,0.220339,0.101562,0.139037,128.000000
^2,0.000000,0.000000,0.000000,7.000000
^g,0.000000,0.000000,0.000000,1.000000
^h,0.000000,0.000000,0.000000,6.000000
^q,0.000000,0.000000,0.000000,10.000000
aa,0.269231,0.069307,0.110236,101.000000
aap_am,0.000000,0.000000,0.000000,1.000000
ad,0.000000,0.000000,0.000000,11.000000
ar,0.000000,0.000000,0.000000,4.000000
arp_nd,0.000000,0.000000,0.000000,3.000000


## 95 percentile pooling

In [53]:
train = makedataset("95per",df_train)
test =  makedataset("95per",df_test)

In [54]:
train_utterances=train.iloc[:,-7:-1]
y_train=train.iloc[:,-1]
test_utterances=test.iloc[:,-7:-1]
y_test=test.iloc[:,-1]

In [55]:
model = LogisticRegression()
model.fit(train_utterances, y_train)
pred = model.predict(test_utterances)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWar

,precision,recall,f1-score,support
%,0.000000,0.000000,0.000000,128.000000
^2,0.000000,0.000000,0.000000,7.000000
^g,0.000000,0.000000,0.000000,1.000000
^h,0.000000,0.000000,0.000000,6.000000
^q,0.000000,0.000000,0.000000,10.000000
aa,0.000000,0.000000,0.000000,101.000000
aap_am,0.000000,0.000000,0.000000,1.000000
ad,0.000000,0.000000,0.000000,11.000000
ar,0.000000,0.000000,0.000000,4.000000
arp_nd,0.000000,0.000000,0.000000,3.000000


In [56]:
model = LinearSVC(class_weight='balanced')

model.fit(train_utterances, y_train)
pred = model.predict(test_utterances)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

,precision,recall,f1-score,support
%,0.571429,0.031250,0.059259,128.000000
^2,0.000000,0.000000,0.000000,7.000000
^g,0.000000,0.000000,0.000000,1.000000
^h,0.000000,0.000000,0.000000,6.000000
^q,0.000000,0.000000,0.000000,10.000000
aa,0.000000,0.000000,0.000000,101.000000
aap_am,0.000000,0.000000,0.000000,1.000000
ad,0.000000,0.000000,0.000000,11.000000
ar,0.000000,0.000000,0.000000,4.000000
arp_nd,0.000000,0.000000,0.000000,3.000000


In [57]:
from sklearn import tree

model = tree.DecisionTreeClassifier()
model.fit(train_utterances, y_train)
pred = model.predict(test_utterances)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

,precision,recall,f1-score,support
%,0.135135,0.117188,0.125523,128.000000
^2,0.000000,0.000000,0.000000,7.000000
^g,0.000000,0.000000,0.000000,1.000000
^h,0.250000,0.166667,0.200000,6.000000
^q,0.000000,0.000000,0.000000,10.000000
aa,0.062500,0.059406,0.060914,101.000000
aap_am,0.000000,0.000000,0.000000,1.000000
ad,0.000000,0.000000,0.000000,11.000000
ar,0.000000,0.000000,0.000000,4.000000
arp_nd,0.000000,0.000000,0.000000,3.000000


In [58]:
from sklearn.neighbors import KNeighborsClassifier
acc=[]
for k in range(1,50):
    model = KNeighborsClassifier(n_neighbors = k)
    model.fit(train_utterances, y_train)
    pred = model.predict(test_utterances)
    pred = classification_report(y_test, pred, output_dict=True)
    report = pd.DataFrame(pred).transpose()
    acc.append(report.iloc[-3,0])

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [60]:
model = KNeighborsClassifier(n_neighbors = 26)
model.fit(train_utterances, y_train)
pred = model.predict(test_utterances)
pred = classification_report(y_test, pred, output_dict=True)
report = pd.DataFrame(pred).transpose()
report

/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samhitha/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

,precision,recall,f1-score,support
%,0.200000,0.085938,0.120219,128.00000
^2,0.000000,0.000000,0.000000,7.00000
^g,0.000000,0.000000,0.000000,1.00000
^h,0.000000,0.000000,0.000000,6.00000
^q,0.000000,0.000000,0.000000,10.00000
aa,0.217391,0.049505,0.080645,101.00000
aap_am,0.000000,0.000000,0.000000,1.00000
ad,0.000000,0.000000,0.000000,11.00000
ar,0.000000,0.000000,0.000000,4.00000
arp_nd,0.000000,0.000000,0.000000,3.00000


In [59]:
acc

[0.28674431503050474,
 0.26400443704936216,
 0.29728230726566834,
 0.35884636716583473,
 0.39267886855241263,
 0.4054353854686633,
 0.41985579589572936,
 0.4248474764281753,
 0.4276206322795341,
 0.4331669439822518,
 0.4292845257903494,
 0.43594009983361065,
 0.43760399334442596,
 0.4370493621741542,
 0.4420410427066001,
 0.44093178036605657,
 0.4442595673876872,
 0.44315030504714364,
 0.44647809206877426,
 0.4442595673876872,
 0.4459234608985025,
 0.447032723239046,
 0.44647809206877426,
 0.44869661674986133,
 0.4492512479201331,
 0.4514697726012202,
 0.45036051026067664,
 0.45202440377149194,
 0.4509151414309484,
 0.4559068219633943,
 0.4547975596228508,
 0.4531336661120355,
 0.45701608430393786,
 0.4564614531336661,
 0.45535219079312256,
 0.454242928452579,
 0.45535219079312256,
 0.45701608430393786,
 0.45535219079312256,
 0.4564614531336661,
 0.45368829728230725,
 0.454242928452579,
 0.4531336661120355,
 0.45202440377149194,
 0.4509151414309484,
 0.4531336661120355,
 0.452579034941

In [68]:
train.to_csv("train_90.csv")
test.to_csv("test_90.csv")